In [1]:
import pymysql

In [5]:
db = pymysql.connect('localhost','root','new_pass','answer')

In [6]:
cursor = db.cursor()

In [8]:
cursor.execute("SELECT VERSION()")

1

In [9]:
data = cursor.fetchone()
print("Database version: %s " % data)

Database version: 5.7.20 


In [10]:
sql_1 = "SELECT * FROM employees LIMIT 2"
cursor.execute(sql_1)

2

In [11]:
db.close()

In [13]:
db = pymysql.connect('localhost','root','new_pass','testdb')

In [14]:
cursor = db.cursor()

In [16]:
show_tables = "SHOW tables"
cursor.execute(show_tables)
data = cursor.fetchall()
print(data)

(('products',), ('sales',), ('stores',))


In [18]:
desc_products = "DESC products"
cursor.execute(desc_products)
data = cursor.fetchall()
print(data)

(('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('product_name', 'varchar(100)', 'YES', '', None, ''),
 ('price', 'decimal(13,2)', 'YES', '', None, ''))

In [19]:
max_price = "SELECT MAX(price) FROM products"
cursor.execute(max_price)
data = cursor.fetchall()
print(data)

((Decimal('1299.00'),),)


In [23]:
avg_price = "SELECT AVG(price) FROM products"
cursor.execute(avg_price)
data = cursor.fetchall()
print(data)

((Decimal('865.666667'),),)


In [28]:
def func_query(cursor,sql_statement):
    cursor.execute(sql_statement)
    data = cursor.fetchall()
    return data
func_query(cursor,min_price)

((Decimal('865.666667'),),)

In [25]:
drop_if_exists = "DROP TABLE IF EXISTS `employee`"
create_table_employee = '''
CREATE TABLE `employee` (`id` INT(10) NOT NULL AUTO_INCREMENT,
`first_name` CHAR(20) NOT NULL, `last_name` CHAR(20) DEFAULT NULL,
`age` INT(11) DEFAULT NULL, `sex` CHAR(1) DEFAULT NULL,
`income` float DEFAULT NULL, PRIMARY KEY (`id`)) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''
cursor.execute(drop_if_exists)

D:\Continuum\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1051, "Unknown table 'testdb.employee'")
  result = self._query(query)


0

In [26]:
cursor.execute(create_table_employee)

0

In [30]:
func_query(cursor,"SHOW TABLES;")

(('employee',), ('products',), ('sales',), ('stores',))

In [31]:
func_query(cursor,"DESC employee;")

(('id', 'int(10)', 'NO', 'PRI', None, 'auto_increment'),
 ('first_name', 'char(20)', 'NO', '', None, ''),
 ('last_name', 'char(20)', 'YES', '', None, ''),
 ('age', 'int(11)', 'YES', '', None, ''),
 ('sex', 'char(1)', 'YES', '', None, ''),
 ('income', 'float', 'YES', '', None, ''))

In [39]:
sql_insert = '''
INSERT INTO employee(
first_name,last_name,age,sex,
income) VALUES ('XiuYue','Wang',28,
'M',20000.00)
'''
# 大坑，执行INSERT 语句后，未执行commit
# 数据表中插入操作，并未真正提交，后面查询不出来。
cursor.execute(sql_insert)

1

In [40]:
func_query(cursor,"SELECT * FROM employee WHERE first_name='XiuYue'")

((1, 'XiuYue', 'Wang', 28, 'M', 20000.0),)

In [43]:
sql_insert_1 = '''
INSERT INTO employee(first_name,last_name,age,
sex,income) VALUES ('WenSheng','Xu',27,'M',15000.00)
'''

try:
    cursor.execute(sql_insert_1)
    db.commit()
except:
    db.rollback()

In [44]:
func_query(cursor,
          "SELECT * FROM employee WHERE first_name='WenSheng'")

((2, 'WenSheng', 'Xu', 27, 'M', 15000.0),)

**动态创建SQL查询**

In [50]:
sql_insert_paras = "INSERT INTO employee(first_name,\
last_name,age,sex,income) \
VALUES ('%s','%s','%d','%c','%f')" % ('Tao','Li',26,'M',18000.00)

In [51]:
try:
    cursor.execute(sql_insert_paras)
    db.commit()
    print("insert successful")
except:
    db.rollback()

insert successful


In [52]:
func_query(cursor,
          "SELECT * FROM employee WHERE first_name='Tao'")

((3, 'Tao', 'Li', 26, 'M', 18000.0),)

In [53]:
first_name = 'HanLiang'
last_name = 'Shen'
age = 26
sex = 'M'
income = 27000.00
sql_insert_paras = "INSERT INTO employee(first_name,\
last_name,age,sex,income) \
VALUES ('%s','%s','%d','%c','%f')" 
try:
    cursor.execute(sql_insert_paras % (first_name,last_name,age,sex,income))
    db.commit()
    print('insert successful')
except:
    db.rollback()

insert successful


In [54]:
func_query(cursor,"SELECT * FROM employee")

((2, 'WenSheng', 'Xu', 27, 'M', 15000.0),
 (3, 'Tao', 'Li', 26, 'M', 18000.0),
 (4, 'HanLiang', 'Shen', 26, 'M', 27000.0))

In [55]:
func_query(cursor,"SELECT * FROM employee")

((2, 'WenSheng', 'Xu', 27, 'M', 15000.0),
 (3, 'Tao', 'Li', 26, 'M', 18000.0),
 (4, 'HanLiang', 'Shen', 26, 'M', 27000.0))

In [57]:
db.close()

**读取操作**

任何数据库上的读操作表示要从数据库中读取获取一些有用的信息。

在建立数据库连接后，就可以对此数据库进行查询了。 可以使用fetchone()方法获取单条记录或fetchall()方法从数据库表中获取多个值。

- fetchone() - 它获取查询结果集的下一行。 结果集是当使用游标对象来查询表时返回的对象。


- fetchall() - 它获取结果集中的所有行。 如果已经从结果集中提取了一些行，则从结果集中检索剩余的行。


- rowcount - 这是一个只读属性，并返回受execute()方法影响的行数。


示例

以下过程查询EMPLOYEE表中所有记录的工资超过1000员工记录信息


In [58]:
db = pymysql.connect('localhost','root','new_pass','answer')

In [59]:
cursor = db.cursor()

In [60]:
func_query(cursor,'SHOW tables;')

(('customers',),
 ('employees',),
 ('items',),
 ('offices',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('productlines',),
 ('products',),
 ('tokens',))

In [61]:
func_query(cursor,"DESC employees;")

(('employeeNumber', 'int(11)', 'NO', 'PRI', None, ''),
 ('lastName', 'varchar(50)', 'NO', '', None, ''),
 ('firstName', 'varchar(50)', 'NO', '', None, ''),
 ('extension', 'varchar(10)', 'NO', '', None, ''),
 ('email', 'varchar(100)', 'NO', '', None, ''),
 ('officeCode', 'varchar(10)', 'NO', 'MUL', None, ''),
 ('reportsTo', 'int(11)', 'YES', 'MUL', None, ''),
 ('jobTitle', 'varchar(50)', 'NO', '', None, ''))

In [62]:
func_query(cursor, "SELECT DISTINCT jobTitle FROM employees;")

(('President',),
 ('VP Sales',),
 ('VP Marketing',),
 ('Sales Manager (APAC)',),
 ('Sale Manager (EMEA)',),
 ('Sales Manager (NA)',),
 ('Sales Rep',))

In [64]:
sql_query = '''
SELECT CONCAT(firstname,' ',lastname) AS `fullname` 
FROM employees WHERE jobTitle='VP Sales';
'''
func_query(cursor,sql_query)    

(('Mary Patterson',),)

In [65]:
db.close()

In [ ]:
db = pymysql.connect('localhost')